<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/main/notebooks/data_preprocessing_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
# use gcs fuse to access google cloud storage
# https://stackoverflow.com/a/60450255/5755604
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# mount google cloud stoarge as drive
!mkdir gcs
!gcsfuse thesis-bucket-option-trade-classification gcs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  43741      0 --:--:-- --:--:-- --:--:-- 43741
OK
34 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be upgraded:
  gcsfuse
1 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 3,120 kB of additional disk space will be used.
(Reading database ... 123947 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.8_amd64.deb ...
Unpacking gcsfuse (0.41.8) over (0.41.7) ...
Setting up gcsfuse (0.41.8) ...
mkdir: cannot create directory ‘gcs’: File exists
2022/10/28 09:24:31.044597 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" usi

In [78]:
import glob

import modin.pandas as pd
import modin.config as cfg
from modin.config import ProgressBar
from distributed import Client
cfg.Engine.put("dask")
ProgressBar.enable()

from tqdm.notebook import tqdm


In [79]:
from google.colab import auth
auth.authenticate_user()


In [80]:
client = Client()

files = glob.glob("./gcs/data/preprocessed/matched_ise_quotes_min_mem_usage_part_*.parquet")

columns = ['UNDERLYING_SYMBOL', 'QUOTE_DATETIME', 'SEQUENCE_NUMBER', 'ROOT',
       'EXPIRATION', 'STRK_PRC', 'OPTION_TYPE', 'TRADE_SIZE', 'TRADE_PRICE',
       'BEST_BID', 'BEST_ASK', 'order_id', 'ask_ex', 'bid_ex', 'bid_size_ex',
       'ask_size_ex', 'price_all_lead', 'price_all_lag', 'optionid',
       'day_vol', 'price_ex_lead', 'price_ex_lag', 'buy_sell']

df = pd.read_parquet(files, columns=columns)

Perhaps you already have a cluster running?
Hosting the HTTP server on port 41327 instead


In [81]:
df.memory_usage(deep=True).sum()

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

5024284470

In [82]:
df.head().T

,0,1,2,3,4
UNDERLYING_SYMBOL,BRCD,SYMC,SPY,ADRX,ORCL
QUOTE_DATETIME,2005-05-02 09:30:02,2005-05-02 09:30:03,2005-05-02 09:30:03,2005-05-02 09:30:03,2005-05-02 09:30:03
SEQUENCE_NUMBER,72515,65366,65373,79195,95870
ROOT,YNU,SYQ,SWG,QAX,ORQ
EXPIRATION,2006-01-21 00:00:00,2005-06-18 00:00:00,2005-05-21 00:00:00,2005-06-18 00:00:00,2005-12-17 00:00:00
STRK_PRC,2.5,15.0,105.0,25.0,14.0
OPTION_TYPE,C,C,C,C,C
TRADE_SIZE,10,10,50,10,15
TRADE_PRICE,2.05,3.9,11.2,0.2,0.25
BEST_BID,1.9,3.6,11.1,0.0,0.3


In [83]:
# check against some stats from sub panel A.1 in Grauer et al
# TODO: convert to proper tests

# trade size
stats_trade_size = df['TRADE_SIZE'].agg(['mean','median','std'])

# moneyness; price underlying / strike
# TODO: Request price for underlyings?

# time to maturity
stats_time_to_maturity = (df['EXPIRATION'] - df['QUOTE_DATETIME']).dt.days
stats_time_to_maturity = stats_time_to_maturity.agg(['mean','median','std'])

# no of observations
stats_n = len(df)

# trade_size = quote size; TRADE_SIZE
stats_trades_with_quote_size_bid = df['bid_size_ex'].eq(df['TRADE_SIZE'])
stats_trades_with_quote_size_ask = df['ask_size_ex'].eq(df['TRADE_SIZE'])
# either ask or bid must be equal, but not both (XOR)
stats_trade_with_quote_size = (stats_trades_with_quote_size_bid ^ stats_trades_with_quote_size_ask).sum() / stats_n

# no of buys
stats_buy_trades = df['buy_sell'].ge(0).sum() / stats_n

# underlyings per day
# stats_underlyings_per_day = df.groupby(['UNDERLYING_SYMBOL','QUOTE_DATETIME']).count().agg(['mean','median','std'])

In [84]:
stats_trade_size

mean      13.615033
median     4.000000
std       77.752615
Name: TRADE_SIZE, dtype: float64

In [85]:
stats_time_to_maturity

mean      106.286479
median     45.000000
std       150.079427
dtype: float64

In [86]:
stats_n

49203747

In [87]:
# TODO: mismatch Grauer et. al report 22.28 %
stats_trades_with_quote_size

0.10956509064238543

In [88]:
stats_buy_trades

0.4745849741890592

In [89]:
df.tail()

,UNDERLYING_SYMBOL,QUOTE_DATETIME,SEQUENCE_NUMBER,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,...,bid_ex,bid_size_ex,ask_size_ex,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,buy_sell
49203742,SVXY,2017-05-31 16:12:19,225274471,SVXY,2017-06-02,152.0,C,1,3.54,2.72,...,2.72,20.0,10.0,4.75,3.00,115528016.0,1.0,5.24,3.85,-1
49203743,SPY,2017-05-31 16:12:45,190004593,SPY,2017-12-15,236.0,P,6,7.27,7.22,...,7.19,753.0,1250.0,7.20,7.61,113308776.0,6.0,5.37,9.01,1
49203744,^NDX,2017-05-31 16:13:39,35914334,NDX,2017-06-02,5690.0,P,12,1.00,0.40,...,0.00,0.0,13.0,0.47,1.40,115919712.0,83.0,0.47,1.40,-1
49203745,DIA,2017-05-31 16:14:02,72897705,DIA,2017-06-02,212.0,C,2,0.05,0.03,...,0.00,0.0,22.0,0.03,0.01,115563008.0,2.0,0.06,0.14,1
49203746,DIA,2017-05-31 16:14:02,72897705,DIA,2017-06-09,212.5,C,2,0.16,0.10,...,0.05,75.0,75.0,0.12,0.14,115722392.0,2.0,0.14,0.15,-1


## create subsample 🔢

In [90]:
year = 2017 
df_sub = df[df['QUOTE_DATETIME'].dt.year == year]
df_sub.to_parquet(f"./gcs/data/preprocessed/matched_ise_quotes_{year}.parquet")

In [91]:
year = 2015
df_sub = df[df['QUOTE_DATETIME'].dt.year == year]
df_sub.to_parquet(f"./gcs/data/preprocessed/matched_ise_quotes_{year}.parquet")